In [ ]:

!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes
!pip install --upgrade langchain  -q
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.7 MB/s eta 0:00:00
     ━━

In [ ]:
!pip install sentence_transformers > /dev/null

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap
import requests
from bs4 import BeautifulSoup
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pinecone

model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

#Functions to prepare data from pdf
The following functions are used to load the pdf and filter out Amharic characters.

In [ ]:
!pip install gitpython
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.3 MB/s eta 0:00:00


Loading the file containing the laws

In [ ]:
import git

repo_url = "https://github.com/Yagth/SemanticSearch.git"
local_path = "/content/laws"

git.Repo.clone_from(repo_url, local_path)

<git.repo.base.Repo '/content/laws/.git'>

In [70]:
repo = git.Repo('/content/laws/')
repo.git.pull()

'Updating f692e9c..ff1caf1\nFast-forward\n templates/index.html | 10 +++++-----\n 1 file changed, 5 insertions(+), 5 deletions(-)'

copy the templates file from the repo to the root folder to be available for the flask server

In [71]:
%cp -r laws/templates /content/

In [ ]:
import re
import PyPDF2

def loadASCIIFromPDF(file_path):
    # Load the PDF file
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

        cleaned_text = ""
        # Iterate over each page in the PDF
        for page in pdf_reader.pages:
            text = page.extract_text()

            # Remove Amharic characters using regex
            text = re.sub(r'[\u1200-\u137F\u1380-\u139F\u2D80-\u2DDF\uAB00-\uAB2F\uAB30-\uAB6F\uAB70-\uABBF]', '', text)

            # Remove non printable characters
            text = re.sub(r'[^\x20-\x7E]+', ' ', text)

            # Remove repeated space characters
            text = re.sub(r'\s+', ' ', text)

            # Remove headers of each page.
            text = re.sub(r" gA d L U T Uz q _R .M Federal Negarit Gazette No. 33, 23rd July, 2021 .page \d+[/)(-_.]+|gA d L U T Uz q _R .M Federal Negarit Gazette No. 33, 23rd July, 2021 .page \d+", "", text)
            cleaned_text += "\n" + text

        # Print the cleaned text
        return cleaned_text

laws = loadASCIIFromPDF("/content/laws/Capital Market Proclamation .pdf")


# Vector Database setup
set up pinecone vector database and function to insert data along with a method to insert the datas in chunks

In [65]:
pinecone.init(api_key="8685f4c4-0e39-44e4-b48c-b4357e1de8af", environment="northamerica-northeast1-gcp")
index = pinecone.Index("laws")
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [60]:
def split_text_into_chunks(plain_text, max_chars=2000):
    text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = max_chars,           # Usually chunk sizes are much larger than this
      chunk_overlap  = 20,        # Overlap is needed incase the text is split in odd places
      length_function = len,
    )
    chunks = text_splitter.split_text(plain_text)
    return chunks

def addData(corpusData):
    id  = index.describe_index_stats()['total_vector_count']
    step = min(100, len(corpusData)) #The number of vectors to insert into the database at the same time
    records = []
    for i in range(len(corpusData)):
        chunk=corpusData[i]
        chunkInfo=(str(id+i),
                model.embed_query(chunk),
                {'context': chunk})
        records.append(chunkInfo)
        if len(records) >= step:
          index.upsert(vectors=records)
          print('Batch no. ' + str(len(records)))
          records = []

def addMessage(messages, metadatas):
    id  = index.describe_index_stats()['total_vector_count']
    step = min(100, len(messages)) #The number of vectors to insert into the database at the same time
    records = []
    for i in range(len(messages)):
        chunk= messages[i]
        chunkInfo=(str(id+i),
                model.embed_query(chunk),
                metadatas[i])
        records.append(chunkInfo)

        if len(records) >= step:
          index.upsert(vectors=records)
          print('Batch no. ' + str(len(records)))
          records = []


## Run the following to add pdf to database

In [66]:
chunks = split_text_into_chunks(laws)
addData(chunks)

Batch no. 100


In [67]:
def find_match(query,k):
    query_em = model.embed_query(query)
    result = index.query(
      query_em,
      top_k=k,
      include_values=True,
      include_metadata=True
    )

    return [result['matches'][i]['metadata']['context'] for i in range(k)],[result['matches'][i]['score'] for i in range(k)]

def printResult(datas, metadatas, scores):
  for i in range(len(datas)):
    print("Entry: " + str(i))
    print("\tData: " + datas[i])
    print("\tScore: " + str(scores[i]))
    print("\tLink: " + str(metadatas[i]))


#Prompt
Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    return header + context + "\n\n" + query + "\n"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
def create_prompt(context,query):
    # header = "Answer the question as truthfully as possible using only the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    prompt = '''
Give the most relevant response using only the context provided below.
Don't use any other context outside the given context. If the given context isn't enough to give a response, print 'The given context isn't enough to give a valid respone'

###Context:
    {context}
###Question:
    > {query}
###Response:
    '''.format(context=context, query=query)
    # return header +"\nContext:" + context + "\n\nQuestion: " + query + "\n"
    return prompt

def init_model():
      # Load the model
    print("Tokenizing model...")


    tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
    # tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.3")

    print("Creating model...")
    model = LLaMAForCausalLM.from_pretrained(
        "decapoda-research/llama-7b-hf",
        load_in_8bit=True,
        device_map="auto",
    )
    print("Fine tunning model...")
    model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")
    # model = AutoModelForCausalLM.from_pretrained(model, "lmsys/vicuna-7b-v1.3")
    # inputs = tokenizer(
        # prompt,
        # return_tensors="pt",
    # )

    return tokenizer, model

def generate_answer(prompt, tokenizer, model):
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    print("Generating config model...")
    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    # for s in generation_output.sequences:
        # print(tokenizer.decode(s))
    return (tokenizer.decode(generation_output.sequences[0]))


In [ ]:
tokenizer, modelNew = init_model()

Tokenizing model...


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Creating model...


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

Fine tunning model...


In [ ]:
question = "What is decided about LLM ?"
metadatas, datas, scores = find_match(question,5)
printResult(datas, metadatas, scores)

In [ ]:
combinedData = [[data, metadata] for data, metadata, score in zip(datas, metadatas, scores) if score > 0.2 and metadata['ispublic']]
context = "\n".join(""+data[0] + "\nBy-> "+data[1]['name'] +"  date-> "+str(data[1]['time'])+"\n" for data in combinedData)
if context == "":
  context = "Not enough context"
print(context)


In [ ]:
prompt = create_prompt(context, question)
response = generate_answer(prompt, tokenizer, modelNew)
lines = response.split("\n")
response = lines[len(lines) - 1].strip()
print(prompt)
print(response)

# Web appliation server for the model
This section involves creating a web server application using falsk that will host it externally using ngrok to be accessed globally.

In [ ]:
!pip install flask-ngrok

Installing ngrok

In [ ]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=a368abcaa702128c603750062ba1bba3b24a45651e220e9d9951e9a1f3b05363
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


The following cell is used to set up the ngrok with the environment variable need to run it. The xxxx is a place holder for the authentication token that can be found from the ngrok website

In [ ]:
!ngrok authtoken "Authtoken"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, request, render_template, jsonify
from flask_ngrok import run_with_ngrok

In [ ]:
app = Flask(__name__)

run_with_ngrok(app)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/generateResponse',methods=['POST'])
def generateResponse():
    question = request.form['searchStr']

    metadatas, scores = find_match(question,10)

    contexts = []

    for i, context in enumerate(metadatas):
      if scores[i] > 0.2:
        contexts.append(context)

    "".join(contexts)

    if len(contexts) == 0:
      context = "Not enough context"

    prompt = create_prompt(context, question)
    response = generate_answer(prompt, tokenizer, modelNew)
    lines = response.split("\n")
    response = lines[len(lines) - 1].strip()
    print(prompt)

    data = {
        "response": response,
        "context": context,
    }

    return jsonify(data)

app.run()